In [1]:
import pandas as pd
import numpy as np

from datetime import date
#import calendar

import xgboost

#import sqlite3 as sql

In [2]:
#import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV 
#from scipy.stats import skew
#from collections import OrderedDict

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import VotingRegressor

In [4]:
#Import cleaned training data
train = pd.read_csv('train.csv')

In [5]:
print(train.head())

   id  day  month  year  hour      speed
0   0    1      1  2017     0  43.002930
1   1    1      1  2017     1  46.118696
2   2    1      1  2017     2  44.294158
3   3    1      1  2017     3  41.067468
4   4    1      1  2017     4  46.448653


In [6]:
train.dtypes

id         int64
day        int64
month      int64
year       int64
hour       int64
speed    float64
dtype: object

In [7]:
train.shape

(14006, 6)

In [8]:
#Create a column for date
train['date']=pd.to_datetime(train[['month', 'day', 'year']])

In [9]:
#Create a column for weekday
train['weekday']=train['date'].dt.day_name()

In [10]:
#Import rainfall data from Hong Kong Observatory
rainfall = pd.read_csv('Rainfall.csv')

In [11]:
rainfall

,year,month,day,rainfall
0,2017,1,1,0.0
1,2017,1,2,0.0
2,2017,1,3,0.0
3,2017,1,4,0.0
4,2017,1,5,0.0
...,...,...,...,...
725,2018,12,27,0.0
726,2018,12,28,0.0
727,2018,12,29,0.0
728,2018,12,30,0.0


In [12]:
#Import data of public holiday
ph = pd.read_csv('PublicHoliday.csv')

In [13]:
#Import Typhoon data
typhoon = pd.read_csv('Typhoon.csv')

In [14]:
#Map the data by year, month & day
train1 = pd.merge(train, rainfall, how='left', on=['year', 'month','day'])

In [15]:
train1.shape

(14006, 9)

In [16]:
train2 = pd.merge(train1, ph, how='left', on=['year', 'month','day'])

In [17]:
train2.shape

(14006, 13)

In [18]:
train2[train2.holiday=='Yes']

,id,day,month,year,hour,speed,date,weekday,rainfall,holiday,school_holiday,early_release,long_vacation
24,24,2,1,2017,0,47.621563,2017-01-02,Monday,0.0,Yes,Yes,NaN,NaN
25,25,2,1,2017,1,47.070161,2017-01-02,Monday,0.0,Yes,Yes,NaN,NaN
26,26,2,1,2017,2,46.894923,2017-01-02,Monday,0.0,Yes,Yes,NaN,NaN
27,27,2,1,2017,3,47.061395,2017-01-02,Monday,0.0,Yes,Yes,NaN,NaN
28,28,2,1,2017,4,47.016156,2017-01-02,Monday,0.0,Yes,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13937,13937,26,12,2018,13,27.618660,2018-12-26,Wednesday,0.0,Yes,Yes,NaN,NaN
13938,13938,26,12,2018,15,27.932919,2018-12-26,Wednesday,0.0,Yes,Yes,NaN,NaN
13939,13939,26,12,2018,20,37.903486,2018-12-26,Wednesday,0.0,Yes,Yes,NaN,NaN
13940,13940,26,12,2018,21,42.777791,2018-12-26,Wednesday,0.0,Yes,Yes,NaN,NaN


In [19]:
train2['holiday'].fillna(0)

0        0
1        0
2        0
3        0
4        0
        ..
14001    0
14002    0
14003    0
14004    0
14005    0
Name: holiday, Length: 14006, dtype: object

In [20]:
train2.to_csv('train2.csv')

In [21]:
train2.describe()

,id,day,month,year,hour,speed,rainfall
count,14006.00000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000
mean,7002.50000,15.679138,6.491932,2017.375268,11.529202,32.779118,6.788805
std,4043.32827,8.793388,3.435815,0.484209,6.926150,13.573813,23.237484
min,0.00000,1.000000,1.000000,2017.000000,0.000000,2.573417,0.000000
25%,3501.25000,8.000000,4.000000,2017.000000,6.000000,19.301089,0.000000
50%,7002.50000,16.000000,6.000000,2017.000000,12.000000,36.580595,0.000000
75%,10503.75000,23.000000,9.000000,2018.000000,18.000000,45.877665,2.500000
max,14005.00000,31.000000,12.000000,2018.000000,23.000000,53.161286,216.000000


In [22]:
#Import traffic accident & congestion data (Tseung Kwan O)
TKO_traffic=pd.read_csv('Traffic congestion.csv')

In [23]:
TKO_traffic.shape

(345, 12)

In [24]:
hour=[]
year=[]
month=[]
day=[]

for row in range(TKO_traffic.shape[0]):
    for i in range(TKO_traffic.start_hour[row],TKO_traffic.end_hour[row]):
        hour.append(i)
        year.append(TKO_traffic.year[row])
        month.append(TKO_traffic.month[row])
        day.append(TKO_traffic.day[row])

TKO_traffic2 = pd.DataFrame(
    {'year': year,
     'month': month,
     'day': day,
     'hour': hour
    })

In [25]:
TKO_traffic2['TKO_congestion']='Yes'

In [26]:
TKO_traffic2.head()

,year,month,day,hour,TKO_congestion
0,2017,10,31,9,Yes
1,2017,11,3,8,Yes
2,2017,11,7,15,Yes
3,2017,11,7,16,Yes
4,2017,11,18,12,Yes


In [27]:
#Import traffic accident & congestion data (Clear Water Bay)
CWB_traffic=pd.read_csv('Traffic_ClearWaterBay.csv')
CWB_traffic.shape

(59, 10)

In [28]:
hour=[]
year=[]
month=[]
day=[]

for row in range(CWB_traffic.shape[0]):
    for i in range(CWB_traffic.start_hour[row],CWB_traffic.end_hour[row]):
        hour.append(i)
        year.append(CWB_traffic.year[row])
        month.append(CWB_traffic.month[row])
        day.append(CWB_traffic.day[row])

CWB_traffic2 = pd.DataFrame(
    {'year': year,
     'month': month,
     'day': day,
     'hour': hour
    })

In [29]:
CWB_traffic2['CWB_congestion']='Yes'

In [30]:
CWB_traffic2.head()

,year,month,day,hour,CWB_congestion
0,2017,11,3,7,Yes
1,2017,11,13,8,Yes
2,2017,11,29,15,Yes
3,2017,11,29,16,Yes
4,2017,11,30,13,Yes


In [31]:
train2b = pd.merge(train2, TKO_traffic2, how='left', on=['year', 'month','day', 'hour'])
train3 = pd.merge(train2b, CWB_traffic2, how='left', on=['year', 'month','day', 'hour'])

In [32]:
train3['holiday'].fillna(0)
train3['TKO_congestion'].fillna(0)
train3['CWB_congestion'].fillna(0)

0        0
1        0
2        0
3        0
4        0
        ..
14002    0
14003    0
14004    0
14005    0
14006    0
Name: CWB_congestion, Length: 14007, dtype: object

In [33]:
typhoon['typhoon']='Yes'

In [34]:
train4 = pd.merge(train3, typhoon, how='left', on=['year', 'month','day', 'hour'])

In [35]:
train4

,id,day,month,year,hour,speed,date,weekday,rainfall,holiday,...,long_vacation,TKO_congestion,CWB_congestion,intensity,name,signal,direction,typhoon_greater_8,typhoon_off,typhoon
0,0,1,1,2017,0,43.002930,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,2017,1,46.118696,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,1,2017,2,44.294158,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,1,2017,3,41.067468,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,1,2017,4,46.448653,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14008,14001,31,12,2018,12,19.865269,2018-12-31,Monday,0.0,NaN,...,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14009,14002,31,12,2018,15,17.820375,2018-12-31,Monday,0.0,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14010,14003,31,12,2018,16,12.501851,2018-12-31,Monday,0.0,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14011,14004,31,12,2018,18,15.979319,2018-12-31,Monday,0.0,NaN,...,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#Import self-defined grouping on peak
peak=pd.read_csv('Peak.csv')

In [37]:
train5 = pd.merge(train4, peak, how='left', on=['weekday', 'hour'])

In [38]:
train5

,id,day,month,year,hour,speed,date,weekday,rainfall,holiday,...,CWB_congestion,intensity,name,signal,direction,typhoon_greater_8,typhoon_off,typhoon,average_speed,peak
0,0,1,1,2017,0,43.002930,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.821433,4
1,1,1,1,2017,1,46.118696,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.005833,4
2,2,1,1,2017,2,44.294158,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.955793,4
3,3,1,1,2017,3,41.067468,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.094632,4
4,4,1,1,2017,4,46.448653,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.180753,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14008,14001,31,12,2018,12,19.865269,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.216490,2
14009,14002,31,12,2018,15,17.820375,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.796675,1
14010,14003,31,12,2018,16,12.501851,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.740849,1
14011,14004,31,12,2018,18,15.979319,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.186349,1


In [39]:
# daypart function: define different daypart grouping according to hour
def daypart(hour):
    if hour in [2,3,4,5]:
        return "dawn"
    elif hour in [6,7,8,9]:
        return "morning"
    elif hour in [10,11,12,13]:
        return "noon"
    elif hour in [14,15,16,17]:
        return "afternoon"
    elif hour in [18,19,20,21]:
        return "evening"
    else: return "midnight"
# utilize it along with apply method
train5['daypart'] = train5['hour'].apply(daypart)

In [40]:
train5

,id,day,month,year,hour,speed,date,weekday,rainfall,holiday,...,intensity,name,signal,direction,typhoon_greater_8,typhoon_off,typhoon,average_speed,peak,daypart
0,0,1,1,2017,0,43.002930,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.821433,4,midnight
1,1,1,1,2017,1,46.118696,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.005833,4,midnight
2,2,1,1,2017,2,44.294158,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.955793,4,dawn
3,3,1,1,2017,3,41.067468,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.094632,4,dawn
4,4,1,1,2017,4,46.448653,2017-01-01,Sunday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.180753,4,dawn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14008,14001,31,12,2018,12,19.865269,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.216490,2,noon
14009,14002,31,12,2018,15,17.820375,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.796675,1,afternoon
14010,14003,31,12,2018,16,12.501851,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.740849,1,afternoon
14011,14004,31,12,2018,18,15.979319,2018-12-31,Monday,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.186349,1,evening


In [41]:
#Create list of categorial features and create dummies
categorical_features=['year','month','day','hour','weekday','holiday','school_holiday','early_release','long_vacation','TKO_congestion','CWB_congestion','intensity','name','signal','direction','typhoon_greater_8','typhoon_off','typhoon','peak','daypart']
train6 = pd.get_dummies(train5,columns =categorical_features)

In [42]:
train6

,id,speed,date,rainfall,average_speed,year_2017,year_2018,month_1,month_2,month_3,...,peak_1,peak_2,peak_3,peak_4,daypart_afternoon,daypart_dawn,daypart_evening,daypart_midnight,daypart_morning,daypart_noon
0,0,43.002930,2017-01-01,0.0,44.821433,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
1,1,46.118696,2017-01-01,0.0,47.005833,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
2,2,44.294158,2017-01-01,0.0,46.955793,1,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
3,3,41.067468,2017-01-01,0.0,47.094632,1,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
4,4,46.448653,2017-01-01,0.0,47.180753,1,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14008,14001,19.865269,2018-12-31,0.0,20.216490,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
14009,14002,17.820375,2018-12-31,0.0,18.796675,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
14010,14003,12.501851,2018-12-31,0.0,18.740849,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
14011,14004,15.979319,2018-12-31,0.0,12.186349,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [43]:
#Apply the same data manipulation method on cleaned test data
test = pd.read_csv('test_input.csv')
test['date']=pd.to_datetime(test[['month', 'day', 'year']])
test['weekday']=test['date'].dt.day_name()
test1= pd.merge(test, rainfall, how='left', on=['year', 'month','day'])
test2 = pd.merge(test1, ph, how='left', on=['year', 'month','day'])
test2['holiday'].fillna(0)

test2b = pd.merge(test2, TKO_traffic2, how='left', on=['year', 'month','day', 'hour'])
test3 = pd.merge(test2b, CWB_traffic2, how='left', on=['year', 'month','day', 'hour'])
test3['holiday'].fillna(0)
test3['TKO_congestion'].fillna(0)
test3['CWB_congestion'].fillna(0)
test4 = pd.merge(test3, typhoon, how='left', on=['year', 'month','day', 'hour'])
test5 = pd.merge(test4, peak, how='left', on=['weekday', 'hour'])
test5['daypart'] = test5['hour'].apply(daypart)
test6 = pd.get_dummies(test5,columns =categorical_features)

test6["year_2017"]=0

In [44]:
test6.head()

,id,date,rainfall,average_speed,year_2018,month_1,month_2,month_3,month_4,month_5,...,peak_2,peak_3,peak_4,daypart_afternoon,daypart_dawn,daypart_evening,daypart_midnight,daypart_morning,daypart_noon,year_2017
0,0,2018-01-01,0.0,46.850480,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1,1,2018-01-01,0.0,46.478504,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,2,2018-01-01,0.0,28.555960,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,3,2018-01-01,0.0,16.860034,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,2018-01-01,0.0,26.014015,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [45]:
every_column_except_y= [col for col in train6.columns if col not in ['speed','id','date','weekday','holiday','school_holiday','early_release','long_vacation','congestion','intensity','name','signal','direction']]

In [46]:
#Split the dataset into training & testing data
X = train6[every_column_except_y]
y = train6['speed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Linear Regression

In [47]:
linear_reg = LinearRegression()

linear_reg.fit(X_train,y_train)
y_predict=linear_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

13.218436306741529

MSE is not good enough.

# SVM

In [48]:
SVM_reg = svm.SVR()

SVM_reg.fit(X_train,y_train)
y_predict=SVM_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

15.442511411127855

MSE is not good enough.

# Nearest Neighbor

In [49]:
knn_reg = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')

knn_reg.fit(X_train,y_train)
y_predict=knn_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

14.623332467315304

MSE is not good enough

# Decision Trees

In [50]:
tree_reg = tree.DecisionTreeRegressor()

tree_reg.fit(X_train,y_train)
y_predict=tree_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

21.88195617801392

MSE is not good enough

# Random Forest

In [51]:
RF_reg = RandomForestRegressor()

RF_reg.fit(X_train,y_train)
y_predict=RF_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

11.799642083659105

In [52]:
RF_reg.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [53]:
#Tune Parameters
#cv_params = {'n_estimators': [ 280, 290, 300]}
#cv_params = {'max_features': [ 35, 40, 45]}
#cv_params = {'max_depth': [ 15, 20, 25, 30]}
cv_params = {'min_samples_leaf': [2, 3 ,4]}

other_params = { 
                  'n_estimators': 280
                , 'max_features': 40
                , 'max_depth': 20
                #, 'min_samples_leaf':2
}
RF_reg = RandomForestRegressor(**other_params)
optimized_RF = GridSearchCV(estimator=RF_reg, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1, n_jobs=4)
optimized_RF.fit(X_train, y_train)

print('Best Parameter：{0}'.format(optimized_RF.best_params_))


Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  4.4min finished


Best Parameter：{'min_samples_leaf': 2}


In [54]:
# Selected
RF_reg = RandomForestRegressor(n_estimators=280, max_features=40, max_depth=20, min_samples_leaf=2)

RF_reg.fit(X_train,y_train)
y_predict=RF_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

11.237499821509541

# Gradient Tree Boosting

In [55]:
#Create Gradient Tree Boosting Model
GTB_reg = GradientBoostingRegressor()

GTB_reg.fit(X_train,y_train)
y_predict=GTB_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

11.589842409314892

In [56]:
GTB_reg.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [57]:
#Tune parameters
#cv_params = {'n_estimators': [325, 350, 375]}
#cv_params = {'max_depth': [3, 4, 5, 6, 7, 8]}
#cv_params = {'subsample': [0.8, 0.9, 0.95, 1]}
cv_params = {'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]}
other_params = {
    #'learning_rate': 0.05
     'n_estimators': 350
    ,'max_depth': 6
   , 'subsample': 0.9
}
GTB_reg = GradientBoostingRegressor(**other_params)
optimized_GTB = GridSearchCV(estimator=GTB_reg, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1, n_jobs=4)
optimized_GTB.fit(X_train, y_train)

print('Best Params：{0}'.format(optimized_GTB.best_params_))

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 10.4min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed: 12.3min finished


Best Params：{'learning_rate': 0.05}


In [58]:
GTB_reg = GradientBoostingRegressor(learning_rate=0.05, n_estimators=350, max_depth=6, subsample=0.9)

GTB_reg.fit(X_train,y_train)
y_predict=GTB_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

10.722341911742728

# XGBoost

In [59]:
XGB_reg = xgboost.XGBRegressor()

XGB_reg.fit(X_train,y_train)
y_predict=XGB_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

[02:39:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


11.55701933542305

In [60]:
#cv_params = {'n_estimators': [245, 250, 255]}
#cv_params = {'max_depth': [9, 10, 11, 12], 'min_child_weight': [4, 5, 6]}
#cv_params = {'gamma': [0.25, 0.3, 0.35]}
#cv_params = {'subsample': [0.8, 0.9, 0.95], 'colsample_bytree': [0.8, 0.9, 0.95]}
#cv_params = {'reg_alpha': [ 4, 5, 6], 'reg_lambda': [ 1, 2, 3]}
cv_params = {'learning_rate': [ 0.06, 0.07, 0.08 ]}

other_params = {
                 #'learning_rate': 0.07
                 'n_estimators': 250
                , 'max_depth': 10
                , 'min_child_weight': 5
                , 'seed': 0
                , 'subsample': 0.9
                , 'colsample_bytree': 0.9
                , 'gamma': 0.3
                , 'reg_alpha': 5
                , 'reg_lambda': 2
               }
model = xgboost.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_squared_error', cv=KFold(n_splits=10, random_state=0, shuffle=True), verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)

print('Best Parameter：{0}'.format(optimized_GBM.best_params_))
print('Best Score:{0}'.format(optimized_GBM.best_score_))

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  8.8min finished


[02:48:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best Parameter：{'learning_rate': 0.07}
Best Score:-11.463912060768982


In [61]:
#selected
XGB_reg = xgboost.XGBRegressor(
                 colsample_bytree=0.9,
                 gamma=0.3,                 
                 learning_rate=0.07,
                 max_depth=10,
                 min_child_weight=5,
                 n_estimators=250,                                                                    
                 reg_alpha=5,
                 reg_lambda=2,
                 subsample=0.9,
                 seed=0,
                 objective ='reg:squarederror') 

XGB_reg.fit(X_train,y_train)
y_predict=XGB_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

10.607502721774143

# Neural Network

In [62]:
nn_reg = MLPRegressor(random_state=1, max_iter=500)

nn_reg.fit(X_train,y_train)
y_predict=nn_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

14.046697703866132

In [63]:
nn_reg.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [64]:
nn_reg = MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes=(35,), activation='relu', alpha = 0.001)

nn_reg.fit(X_train,y_train)
y_predict=nn_reg.predict(X_test)

mean_squared_error(y_test, y_predict)

10.801159422600893

# Selected Best Model

In [65]:
# Select RF
RF_reg_selected = RandomForestRegressor(n_estimators=280, max_features=40, max_depth=20, min_samples_leaf=2)

RF_reg_selected.fit(X_train,y_train)
y_predict=RF_reg_selected.predict(X_test)

mean_squared_error(y_test, y_predict)

11.224890009668846

In [66]:
#Selected GTB
GTB_reg_selected  = GradientBoostingRegressor(learning_rate=0.05, n_estimators=350, max_depth=6, subsample=0.9)

GTB_reg_selected .fit(X_train,y_train)
y_predict=GTB_reg_selected .predict(X_test)

mean_squared_error(y_test, y_predict)

10.806105109385252

In [67]:
#Selected XGB

XGB_reg_selected = xgboost.XGBRegressor(
                 colsample_bytree=0.9,
                 gamma=0.3,                 
                 learning_rate=0.07,
                 max_depth=10,
                 min_child_weight=5,
                 n_estimators=250,                                                                    
                 reg_alpha=5,
                 reg_lambda=2,
                 subsample=0.9,
                 seed=0,
                 objective ='reg:squarederror') 

XGB_reg_selected.fit(X_train,y_train)
y_predict=XGB_reg_selected.predict(X_test)

mean_squared_error(y_test, y_predict)

10.607502721774143

In [68]:
# Selected NN
nn_reg_selected = MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes=(35,), activation='relu', alpha = 0.001)

nn_reg_selected.fit(X_train,y_train)
y_predict=nn_reg_selected.predict(X_test)

mean_squared_error(y_test, y_predict)

10.801159422600893

# Voting Regressor

In [69]:
Vot_reg_selected_2 = VotingRegressor(estimators=[('RF', RF_reg_selected)
                                      , ('GTB', GTB_reg_selected)
                                      , ('XGB', XGB_reg_selected)
                                      , ('NN', nn_reg_selected)]
                            , weights=[.15, .25, .35, .25])

Vot_reg_selected_2.fit(X_train,y_train)
y_predict=Vot_reg_selected_2.predict(X_test)

mean_squared_error(y_test, y_predict)

10.226701434372318

In [70]:
#Selected Voting
Vot_reg_selected = VotingRegressor(estimators=[('GTB', GTB_reg_selected)
                                      , ('XGB', XGB_reg_selected)
                                      , ('NN', nn_reg_selected)]
                            , weights=[.3, .4, .3])

Vot_reg_selected.fit(X_train,y_train)
y_predict=Vot_reg_selected.predict(X_test)

mean_squared_error(y_test, y_predict)

10.179619368331

# Generate prediction with selected model 

In [71]:
train6['predicted_speed']=Vot_reg_selected_2.predict(train6[every_column_except_y])

In [72]:
train6.to_csv('train6.csv')

In [73]:
test6['intensity_Severe Typhoon']=0 
test6['intensity_Tropical Depression']=0 
test6['name_HATO']=0 
test6['name_KHANUN']=0 
test6['name_MAWAR']=0 
test6['name_MERBOK']=0 
test6['name_PAKHAR']=0 
test6['name_ROKE']=0 
test6['name_no name']=0 
test6['signal_9.0']=0 
test6['direction_NW']=0 
test6['direction_SW']=0 
test6['typhoon_off_Yes']=0 
test6['typhoon_pre_off_Yes']=0 

In [74]:
test6['speed']=Vot_reg_selected_2.predict(test6[every_column_except_y])

In [75]:
print(test6.shape)

(3504, 128)


In [76]:
id=0
test6['keep']=bool(test6.id[row]==id+1)

for row in range(test6.shape[0]):
    #print(test5.id[row])
    #print(id)
    test6['keep'] = bool(test6.id[row]==id+1)
    id=test6.id[row]

In [77]:
sum(test6['keep']==False)

0

In [78]:
test7=test6[test6['keep']==True]

In [79]:
print(test6.shape)

(3504, 129)


In [80]:
test7.to_csv('test7.csv')